In [19]:
import pandas as pd
import numpy as np
import ast
import math
import re
import os

In [20]:
# Feature extraction
# Function to calculate first WAP
def calc_wap1(df):
    wap = (df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1']) / (df['bid_size1'] + df['ask_size1'])
    return wap

# Function to calculate second WAP
def calc_wap2(df):
    wap = (df['bid_price2'] * df['ask_size2'] + df['ask_price2'] * df['bid_size2']) / (df['bid_size2'] + df['ask_size2'])
    return wap

In [21]:
#Function used to calculate log return
def log_return(series):
    # 确保Series中没有空值，并且所有输入都转换为浮点数
    series = series.dropna().astype(float)
    return np.log(series).diff()


# Calculate the realized volatility
def realized_volatility(series):
    return np.sqrt(np.sum(series**2))

In [22]:
def Feature_extraction(df):
    data_dict = {}
    # Calculate Wap
    data_dict['wap1'] = calc_wap1(df)
    data_dict['wap2'] = calc_wap2(df)
    data_dict['log_return1'] = np.log(data_dict['wap1']) - np.log(data_dict['wap1'].shift(1))
    data_dict['log_return2'] = np.log(data_dict['wap2']) - np.log(data_dict['wap2'].shift(1))
    # Calculate wap balance
    data_dict['wap_balance'] = abs(data_dict['wap1'] - data_dict['wap2'])
    # Calculate spread
    data_dict['price_spread'] = (df['ask_price1'] - df['bid_price1']) / ((df['ask_price1'] + df['bid_price1']) / 2)
    data_dict['price_spread2'] = (df['ask_price2'] - df['bid_price2']) / ((df['ask_price2'] + df['bid_price2']) / 2)
    data_dict['bid_spread'] = df['bid_price1'] - df['bid_price2']
    data_dict['ask_spread'] = df['ask_price1'] - df['ask_price2']
    #Calculate the spread relative to the average price
    data_dict["bid_ask_spread"] = abs(data_dict['bid_spread'] - data_dict['ask_spread'])
    data_dict['total_volume'] = (df['ask_size1'] + df['ask_size2']) + (df['bid_size1'] + df['bid_size2'])
    #To quantify the current pressure imbalance between buyers and sellers
    data_dict['volume_imbalance'] = abs((df['ask_size1'] + df['ask_size2']) - (df['bid_size1'] + df['bid_size2']))

     # Dict for aggregations
    create_feature_dict = {
        'wap1': [np.sum, np.mean, np.std],
        'wap2': [np.sum, np.mean, np.std],
        'log_return1': [np.sum, realized_volatility, np.mean, np.std],
        'log_return2': [np.sum, realized_volatility, np.mean, np.std],
        'wap_balance': [np.sum, np.mean, np.std],
        'price_spread':[np.sum, np.mean, np.std],
        'price_spread2':[np.sum, np.mean, np.std],
        'bid_spread':[np.sum, np.mean, np.std],
        'ask_spread':[np.sum, np.mean, np.std],
        'total_volume':[np.sum, np.mean, np.std],
        'volume_imbalance':[np.sum, np.mean, np.std],
        "bid_ask_spread":[np.sum, np.mean, np.std],
    }
    
    
    name_dict = {
            np.sum:"sum",
            np.mean:"mean",
            np.std:"std",
            realized_volatility:"realized_volatility"
    }
    


#     # Define window size and step size
#     window_size = 100  # Window size in seconds
#     step_size = 10     # Window step size in seconds

#     # Create a new column 'window_id' to represent the window to which each timestamp belongs
#     df['window_id'] = df['timestamp'].apply(lambda x: x // step_size)

#     # Grouping and aggregation calculations.
#     aggregated_df = df.groupby('window_id').agg(create_feature_dict)

#     # The window ID can be used to calculate the start and end timestamps of each window
#     aggregated_df['window_start'] = aggregated_df.index * step_size
#     aggregated_df['window_end'] = aggregated_df['window_start'] + window_size
    
#     # Reset index so window ID is a column
#     aggregated_df.reset_index(drop=True, inplace=True)

#     df['timestamp'] = pd.to_datetime(df['timestamp']) # 我把这行注释掉了


 # print(data_dict)
    df_t_1 = {}
    for key,value in create_feature_dict.items():
        for func in value:
            df_t_1[key+"_"+name_dict[func]] = func(data_dict[key])

    # print(df_new.index)
    return pd.DataFrame([df_t_1])





In [23]:
def sequentially(df_base):
    lines = []
    max_ts = math.floor(df_base['timestamp'].max())-600
    interval = 600
    interval_step = [100,200,300,400,500]
    for j in range(0,max_ts,10):
        df_t = df_base[(df_base['timestamp'] >= j) & (df_base['timestamp'] < j+interval)]
        # print(df_t)
        df_total = Feature_extraction(df_t.copy())
        # print(df_total)
        df_total['range'] = "{}-{}".format(j,j+600)
            # # print(df_sum)
        lines.append(df_total)
        for step in interval_step:
            df_step = df_t[(df_t['timestamp'] >= j) & (df_t['timestamp'] < j+step)]

            df_step_sum = Feature_extraction(df_step.copy())
            df_step_sum['range'] = "{}-{}".format(j,j+step)
            lines.append(df_step_sum)

    result = pd.concat(lines,axis=0)
    
    return result


In [24]:
def transaction_quantity(df):
    transaction_quantity= df['Total_Size'].diff().fillna(0).abs()
    return transaction_quantity
    
def transaction(df, start_time, end_time):
    # Assuming 'Time' is a column in your dataframe indicating the time of each transaction
    filtered_df = df[(df['Time'] >= start_time) & (df['Time'] < end_time)]
    transaction = filtered_df['Total_Size'].diff() != 0
    return transaction

In [25]:
def tape_Feature_extraction(df, start_time, end_time):
    transaction_quantity_series = transaction_quantity(df)
    transaction_series = transaction(df, start_time, end_time)
    create_feature_dict = {
        'transaction_quantity_sum': transaction_quantity_series.sum(),
        'transaction_count_sum': transaction_series.sum(),
    }
    return pd.DataFrame([create_feature_dict])

In [26]:
def sequentially0(df_base):
    lines = []
    max_ts = math.floor(df_base['Time'].max())-600
    interval = 600
    interval_step = [100,200,300,400,500]
    for j in range(0,max_ts,10):
        df_t = df_base[(df_base['Time'] >= j) & (df_base['Time'] < j+interval)]
        # print(df_t)
        df_total = tape_Feature_extraction(df_t.copy(),j,j+interval)
        # print(df_total)
        df_total['range'] = "{}-{}".format(j,j+600)
            # # print(df_sum)
        lines.append(df_total)
        for step in interval_step:
            df_step = df_t[(df_t['Time'] >= j) & (df_t['Time'] < j+step)]

            df_step_sum = tape_Feature_extraction(df_step.copy(),j,j+step)
            df_step_sum['range'] = "{}-{}".format(j,j+step)
            lines.append(df_step_sum)

    result = pd.concat(lines,axis=0)
    
    return result

In [31]:
lob_data=pd.read_csv("./JPMorgan_Set01/LOBs_test/Clean_UoB_Set01_2025-01-02LOBs.txt") 
sequentially(lob_data).to_csv("temp.csv",index=False)

In [32]:
df=pd.read_csv("temp.csv")

# Set loop value
cycle_values = [600, 100, 200, 300, 400, 500]

# Generate a cyclic sequence whose length is the same as the number of rows of df
num_rows = len(df)
cycle_sequence = cycle_values * (num_rows // len(cycle_values)) + cycle_values[:num_rows % len(cycle_values)]

# Assign loop sequence to 'range' column
df['range'] = cycle_sequence

# Add an index to each range loop
df['index'] = (df.index // 6) + 1

#Merge rows with the same index into one row
pivot_df = df.pivot(index='index', columns='range')

num_rows = len(pivot_df)

# Create a looping timeline that increments every 10
start_time = [(i * 10)  for i in range(num_rows)]
end_time = [600 + i * 10 for i in range(num_rows)]

# Add timeline to DataFrame
pivot_df['start_time'] = start_time
pivot_df['end_time'] = end_time


pivot_df

wap1_sum                                              \
range            100            200            300           400   
index                                                              
1      284477.421657  598195.836027  914845.478579  1.218197e+06   
2      302972.084211  607660.874988  931769.201631  1.240736e+06   
3      311157.017346  609161.824392  938986.477227  1.243041e+06   
4      305671.523712  613249.917708  945603.745290  1.236985e+06   
5      305350.564285  611610.455502  941129.846615  1.232477e+06   
...              ...            ...            ...           ...   
2996   309473.414292  628084.401817  959314.376866  1.314271e+06   
2997   291596.002540  639213.966227  971535.790755  1.324465e+06   
2998   287609.203310  632726.721733  964429.595057  1.304799e+06   
2999   276943.785345  631900.386385  962136.093042  1.287614e+06   
3000   279742.868498  632994.152288  973639.936693  1.286684e+06   

                                    wap1_mean                          \
range           500           600         100         200         300   
index                                                                   
1      1.498361e+06  1.810106e+06  263.649140  263.754778  264.406208   
2      1.512365e+06  1.820859e+06  262.313493  263.512955  264.256722   
3      1.522377e+06  1.825671e+06  261.916681  263.592308  264.205537   
4      1.515210e+06  1.830422e+06  261.705072  263.423504  263.987645   
5      1.504330e+06  1.833409e+06  262.103489  263.284742  263.843523   
...             ...           ...         ...         ...         ...   
2996   1.588998e+06  1.896714e+06  288.687886  291.725221  291.939859   
2997   1.584228e+06  1.909766e+06  288.708913  291.612211  291.840129   
2998   1.564906e+06  1.904923e+06  289.345275  291.713565  291.720991   
2999   1.554021e+06  1.905068e+06  289.085371  291.870848  291.821684   
3000   1.563688e+06  1.916837e+06  289.289419  292.241068  292.033574   

                   ... bid_ask_spread_mean            bid_ask_spread_std  \
range         400  ...                 500        600                100   
index              ...                                                     
1      264.021810  ...            9.957791   9.246759          23.053364   
2      263.649779  ...            9.871822   9.220243          22.349206   
3      263.356179  ...            9.876665   9.182606          22.074717   
4      263.076320  ...            9.861536   9.154589          22.285791   
5      263.068631  ...            9.876990   9.209981          22.310429   
...           ...  ...                 ...        ...                ...   
2996   292.385062  ...           12.584975  13.111385          28.244816   
2997   292.054039  ...           13.014586  12.935212          27.785927   
2998   291.901333  ...           13.139039  12.754720          23.925653   
2999   291.975907  ...           13.495577  12.676999          22.381048   
3000   292.096322  ...           12.929841  12.239250          15.937555   

                                                             start_time  \
range        200        300        400        500        600              
index                                                                     
1      17.320100  15.355558  19.626748  20.559770  18.828524          0   
2      17.181836  15.213066  19.432295  20.469905  18.774648         10   
3      17.181870  15.155563  19.405147  20.405054  18.760151         20   
4      17.138417  15.097792  19.441116  20.447711  18.743252         30   
5      17.153046  15.136548  20.504223  20.520457  18.731088         40   
...          ...        ...        ...        ...        ...        ...   
2996   21.203196  23.766567  20.728969  22.346349  22.994135      29950   
2997   20.003676  23.060191  20.127990  23.025690  22.614524      29960   
2998   17.501498  21.608425  18.971132  22.801159  21.864980      29970   
2999   15.735321  20.762561  19.664630  23.341765  21.575156      29980

In [37]:
def Feature_extraction_one(file_path,file_path0):
    # Lob data
    df = pd.read_csv(file_path)
    sequentially(df).to_csv("temp.csv",index=False)
    df=pd.read_csv("temp.csv")
    # Set loop value
    cycle_values = [600, 100, 200, 300, 400, 500]

    # Generate a cyclic sequence whose length is the same as the number of rows of df
    num_rows = len(df)
    cycle_sequence = cycle_values * (num_rows // len(cycle_values)) + cycle_values[:num_rows % len(cycle_values)]

    # Assign loop sequence to 'range' column
    df['range'] = cycle_sequence

    # Add an index to each range loop
    df['index'] = (df.index // 6) + 1

    #Merge rows with the same index into one row
    pivot_df = df.pivot(index='index', columns='range')

    num_rows = len(pivot_df)

    # Create a looping timeline that increments every 10
    start_time = [(i * 10)  for i in range(num_rows)]
    end_time = [600 + i * 10 for i in range(num_rows)]

    # Add timeline to DataFrame
    pivot_df['start_time'] = start_time
    pivot_df['end_time'] = end_time
    
    
    # Tape data
    tape_data=pd.read_csv(file_path0)
    sequentially0(tape_data).to_csv("temp.csv",index=False)
    df=pd.read_csv("temp.csv")
    
    cycle_values = [600, 100, 200, 300, 400, 500]

    # Generate a cyclic sequence whose length is the same as the number of rows of df
    num_rows = len(df)
    cycle_sequence = cycle_values * (num_rows // len(cycle_values)) + cycle_values[:num_rows % len(cycle_values)]

    # Assign loop sequence to 'range' column
    df['range'] = cycle_sequence

    # Add an index to each range loop
    df['index'] = (df.index // 6) + 1

    #Merge rows with the same index into one row
    pivot_df1 = df.pivot(index='index', columns='range')

    df_combined = pd.concat([pivot_df1, pivot_df], axis=1)
    df_combined['size_tau2'] = np.sqrt( 1/ df_combined['transaction_count_sum',600] )
    df_combined['size_tau2.1'] = np.sqrt( 0.83/ df_combined['transaction_count_sum',100] )
    df_combined['size_tau2.2'] = np.sqrt( 0.66/ df_combined['transaction_count_sum',200] )
    df_combined['size_tau2.3'] = np.sqrt( 0.5/ df_combined['transaction_count_sum',300] )
    df_combined['size_tau2.4'] = np.sqrt( 0.33/ df_combined['transaction_count_sum',400]  )
    df_combined['size_tau2.5'] = np.sqrt( 0.16/ df_combined['transaction_count_sum',500] )
    
    pattern = r'Clean_UoB_Set.*?\.txt'
    file_name = re.findall(pattern, file_path)[0]
    file_name = "Featured_" + file_name
    
    df_combined.to_csv(file_name,index=False)

In [38]:
def lob_extract_features_all(directory0,directory1):
    files_lob = os.listdir(directory0)
    files_tape = os.listdir(directory1)
    
    for file in zip(files_lob,files_tape):
        file_path0 = os.path.join(directory0, file[0])
        file_path1 = os.path.join(directory1, file[1])
        Feature_extraction_one(file_path0,file_path1)

In [39]:
lob_extract_features_all('./JPMorgan_Set01/LOBs_test/','./JPMorgan_Set01/Tapes_test/')